In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
import torch
import clip
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-hi1z9mo1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-hi1z9mo1
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=3c72d446b1de8594c5bf2f9ca0e4d4a539d9c93b800dda1e9d498b2cef25c34b
  Stored in directory: /tmp/pip-ephem-wheel-cache-tqp1qsi0/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


### Hyperparameters

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

### Dataset

In [3]:
# Load the CUB-200-2011 dataset
def load_cub_dataset(data_dir):
    images = pd.read_csv(os.path.join(data_dir, 'images.txt'), sep=' ', names=['image_id', 'file_path'])
    labels = pd.read_csv(os.path.join(data_dir, 'image_class_labels.txt'), sep=' ', names=['image_id', 'class_id'])
    classes = pd.read_csv(os.path.join(data_dir, 'classes.txt'), sep=' ', names=['class_id', 'class_name'])
    return images, labels, classes
data_dir = '/kaggle/input/cub2002011/CUB_200_2011'

images, labels, classes = load_cub_dataset(data_dir)

print(images.head())
print(labels.head())
print(classes.head())

print(images.shape)
print(labels.shape)
print(classes.shape)

   image_id                                          file_path
0         1  001.Black_footed_Albatross/Black_Footed_Albatr...
1         2  001.Black_footed_Albatross/Black_Footed_Albatr...
2         3  001.Black_footed_Albatross/Black_Footed_Albatr...
3         4  001.Black_footed_Albatross/Black_Footed_Albatr...
4         5  001.Black_footed_Albatross/Black_Footed_Albatr...
   image_id  class_id
0         1         1
1         2         1
2         3         1
3         4         1
4         5         1
   class_id                  class_name
0         1  001.Black_footed_Albatross
1         2        002.Laysan_Albatross
2         3         003.Sooty_Albatross
3         4       004.Groove_billed_Ani
4         5          005.Crested_Auklet
(11788, 2)
(11788, 2)
(200, 2)


## Task 1

### CLIP

In [4]:
clip_model, clip_preprocess = clip.load("ViT-B/32", device=DEVICE, jit=False)
clip_model.eval()

def get_clip_features(img_path):
    img = Image.open(img_path)
    img_input = clip_preprocess(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        img_features = clip_model.encode_image(img_input)
    return img_features

def get_clip_text_features(text):
    text_input = clip.tokenize([text]).to(DEVICE)
    with torch.no_grad():
        text_features = clip_model.encode_text(text_input)
    return text_features

def get_clip_similarity_score(img_features, text_features):
    img_features /= img_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity_score = (100.0 * img_features @ text_features.T).softmax(dim=-1)
    return similarity_score.item()

def get_cosine_similarity(img_features: torch.Tensor, txt_features: torch.Tensor) -> torch.Tensor:
    """
    Computes the cosine similarity between image and text feature tensors.
    
    Parameters:
    img_features (torch.Tensor): Feature tensor for the image.
    txt_features (torch.Tensor): Feature tensor for the text.

    Returns:
    torch.Tensor: Cosine similarity score between -1 and 1.
    """

    img_features = img_features.squeeze()
    txt_features = txt_features.squeeze()
    
    # Compute dot product
    dot_product = torch.dot(img_features, txt_features)
    
    # Compute L2 norms
    norm_img = torch.norm(img_features, p=2)
    norm_txt = torch.norm(txt_features, p=2)
    
    # Avoid division by zero
    if norm_img == 0 or norm_txt == 0:
        return torch.tensor(0.0)  # Handle zero-vector cases
    
    # Compute cosine similarity
    similarity = dot_product / (norm_img * norm_txt)
    
    return similarity.item()

def recognize_bird_species(img_path):
    img_features = get_clip_features(img_path)
    similarities = []
    for class_name in classes['class_name']:
        text_features = get_clip_text_features(class_name)
        similarity_score = get_cosine_similarity(img_features, text_features)
        similarities.append((class_name, similarity_score))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[0][0]

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 149MiB/s]


In [5]:
img_id = 1
image_name = images.iloc[img_id]['file_path']
gt_class = labels.iloc[img_id]['class_id']
gt_class_name = classes[classes['class_id'] == gt_class]['class_name'][0]
bird_species = recognize_bird_species(os.path.join(os.path.join(data_dir, 'images'), image_name))
print(bird_species)
pred_class = classes[classes['class_name'] == bird_species]['class_id'][0]
print(f'Predicted class: {pred_class}, class name: {bird_species}')
print(f"GT class: {gt_class}, class name: {gt_class_name}")

001.Black_footed_Albatross
Predicted class: 1, class name: 001.Black_footed_Albatross
GT class: 1, class name: 001.Black_footed_Albatross


#### Run CLIP on all images

In [20]:
accuracy = 0
with open('clip_predictions.txt', 'w') as f:
    for img_id in tqdm(range(2001, 3000)):
        image_name = images.iloc[img_id]['file_path']
        gt_class = labels.iloc[img_id]['class_id']
        bird_species = recognize_bird_species(os.path.join(os.path.join(data_dir, 'images'), image_name))
        pred_class = classes[classes['class_name'] == bird_species]['class_id'].iloc[0]
        f.write(f'{img_id} {pred_class} {int(gt_class == pred_class)}\n')
        accuracy += int(gt_class == pred_class)
    accuracy /= len(images)
    print(f'Accuracy: {accuracy}')

100%|██████████| 999/999 [28:23<00:00,  1.70s/it]

Accuracy: 0.03579911774686122


In [7]:
def measure_accuracy(file_path):
    accuracy = 0
    nlines = 0
    with open(file_path, 'r') as f:
        for line in f.readlines():
            accuracy += int(line.split(' ')[2])
            nlines += 1
    return accuracy / nlines
accuracy = measure_accuracy('/kaggle/working/clip_predictions.txt')
print(f'Accuracy = {np.round(accuracy*100, 2)}%')

Accuracy = 0.0%
